In [0]:
# activate R magic
%load_ext rpy2.ipython

# Coleta e Transformação de Dados

### Este código contém comandos para filtrar e transformar os dados de aluguel de bikes do nosso dataset.

In [0]:
%%R
# Variável que controla a execução do script
Azure <- FALSE

# Execução de acordo com o valor da variável Azure
if(Azure){
  source("Tools.R")
  bikes <- maml.mapInputPort(1)
  bikes$dteday <- set.asPOSIXct(bikes)
}else{
  source("Tools.R")
  bikes <- read.csv("bikes.csv", sep = ",", header = TRUE, stringsAsFactors = FALSE )

----------------------------------------------------------------------------------------

  # Selecionar as variáveis mais relevantesque serão usadas
    cols <- c("dteday", "mnth", "hr", "holiday",
              "workingday", "weathersit", "temp",
              "hum", "windspeed", "cnt")
  
----------------------------------------------------------------------------------------

  # Criando um subset dos dados
  bikes <- bikes[, cols]
  
  # Transformar o objeto de data - Esta linha gera dois valores NA
  bikes$dteday <- char.toPOSIXct(bikes)
  
  # Esta linha abaixo corrige
  bikes <- na.omit(bikes)

   # Normalizar as variaveis preditoras numericas 
  cols <- c("temp", "hum", "windspeed") 
  bikes[, cols] <- scale(bikes[, cols])  
}

In [6]:
%%R
#?scale
str(bikes)

'data.frame':	17379 obs. of  10 variables:
 $ dteday    : POSIXct, format: "2011-01-01 00:00:00" "2011-01-01 01:00:00" ...
 $ mnth      : int  1 1 1 1 1 1 1 1 1 1 ...
 $ hr        : int  0 1 2 3 4 5 6 7 8 9 ...
 $ holiday   : int  0 0 0 0 0 0 0 0 0 0 ...
 $ workingday: int  0 0 0 0 0 0 0 0 0 0 ...
 $ weathersit: int  1 1 1 1 1 2 1 1 1 1 ...
 $ temp      : num  -1.33 -1.44 -1.44 -1.33 -1.33 ...
 $ hum       : num  0.947 0.896 0.896 0.636 0.636 ...
 $ windspeed : num  -1.55 -1.55 -1.55 -1.55 -1.55 ...
 $ cnt       : int  16 40 32 13 1 1 2 3 8 14 ...


In [0]:
%%R
# Criar uma nova variável para indicar dia da semana (workday)
bikes$isWorking <- ifelse(bikes$workingday & !bikes$holiday, 1, 0)  
# Se as duas condições forem verdadeiras = 1, caso contrário 0. Cria a nova variável bikes$isWorking

In [0]:
%%R
# Adicionar uma coluna com a quantidade de meses, o que vai ajudar a criar o modelo
bikes <- month.count(bikes)

In [0]:
%%R
# Criar um fator ordenado para o dia da semana, comecando por segunda-feira
# Neste fator é convertido para ordenado numérico para ser compativel com os tipos de dados do Azure ML
bikes$dayWeek <- as.factor(weekdays(bikes$dteday))
# Extrair weekdays da coluna dteday e converter para fator. Criar nova variável $dayweek


In [14]:
%%R
# ==> Analise o dataframe bikes. 
# Se os nomes dos dias da semana estiverem em português na coluna bikes$dayWeek, 
# execute o Bloco1 abaixo, caso contrátio, execute o Bloco2 com os nomes em inglês. 
# Execute um bloco ou o outro.
str(bikes$dayWeek)

 Factor w/ 7 levels "Friday","Monday",..: 3 3 3 3 3 3 3 3 3 3 ...


---

# Se o bloco de dias da semana estiver em Português

In [0]:
%%R
# Bloco1
# Se o seu sistema operacional estiver em portugês, execute o comando abaixo.
bikes$dayWeek <- as.numeric(ordered(bikes$dayWeek, 
                                    levels = c("segunda-feira", 
                                               "terça-feira", 
                                               "quarta-feira", 
                                               "quinta-feira", 
                                               "sexta-feira", 
                                               "sábado", 
                                               "domingo")))

# Se o bloco de dias da semana estiver em Inglês

In [0]:
%%R
# Bloco2
# Se o seu sistema operacional estiver em inglês, execute o comando abaixo.
bikes$dayWeek <- as.numeric(ordered(bikes$dayWeek, 
                                    levels = c("Monday", 
                                               "Tuesday", 
                                               "Wednesday", 
                                               "Thursday", 
                                               "Friday", 
                                               "Saturday", 
                                               "Sunday")))


---

In [17]:
%%R
# Agora os dias da semana devem estar como valores numéricos
# Se estiverem como valores NA, volte e verifique se você seguiu as instruções acima.
str(bikes$dayWeek)


 num [1:17379] NA NA NA NA NA NA NA NA NA NA ...


In [18]:
%%R
str(bikes)

'data.frame':	17379 obs. of  13 variables:
 $ dteday    : POSIXct, format: "2011-01-01 00:00:00" "2011-01-01 01:00:00" ...
 $ mnth      : int  1 1 1 1 1 1 1 1 1 1 ...
 $ hr        : int  0 1 2 3 4 5 6 7 8 9 ...
 $ holiday   : int  0 0 0 0 0 0 0 0 0 0 ...
 $ workingday: int  0 0 0 0 0 0 0 0 0 0 ...
 $ weathersit: int  1 1 1 1 1 2 1 1 1 1 ...
 $ temp      : num  -1.33 -1.44 -1.44 -1.33 -1.33 ...
 $ hum       : num  0.947 0.896 0.896 0.636 0.636 ...
 $ windspeed : num  -1.55 -1.55 -1.55 -1.55 -1.55 ...
 $ cnt       : int  16 40 32 13 1 1 2 3 8 14 ...
 $ isWorking : num  0 0 0 0 0 0 0 0 0 0 ...
 $ monthCount: num  1 1 1 1 1 1 1 1 1 1 ...
 $ dayWeek   : num  NA NA NA NA NA NA NA NA NA NA ...


# Feature Engineering | Engenharia de Atributos
## Adicionando novas variáveis 

In [0]:
%%R
# Adiciona uma variável com valores únicos para o horário do dia em dias de semana e dias de fim de semana
# Com isso diferenciamos as horas dos dias de semana, das horas em dias de fim de semana
bikes$workTime <- ifelse(bikes$isWorking, bikes$hr, bikes$hr + 24) 

In [0]:
%%R
# Transforma os valores de hora na madrugada, quando a demanda por bibicletas é praticamente nula 
bikes$xformHr <- ifelse(bikes$hr > 4, bikes$hr - 5, bikes$hr + 19)

In [0]:
%%R
# Adiciona uma variável com valores únicos para o horário do dia para dias de semana e dias de fim de semana
# Considerando horas da madrugada
bikes$xformWorkHr <- ifelse(bikes$isWorking, bikes$xformHr, bikes$xformHr + 24) 

# O trabalho que fizemos até aqui também é chamado de Feature Engineering ou Engenharia de Atributos

In [26]:
%%R
 str(bikes)
# View(bikes)


# Gera saída no Azure ML
if(Azure) maml.mapOutputPort('bikes')

'data.frame':	17379 obs. of  16 variables:
 $ dteday     : POSIXct, format: "2011-01-01 00:00:00" "2011-01-01 01:00:00" ...
 $ mnth       : int  1 1 1 1 1 1 1 1 1 1 ...
 $ hr         : int  0 1 2 3 4 5 6 7 8 9 ...
 $ holiday    : int  0 0 0 0 0 0 0 0 0 0 ...
 $ workingday : int  0 0 0 0 0 0 0 0 0 0 ...
 $ weathersit : int  1 1 1 1 1 2 1 1 1 1 ...
 $ temp       : num  -1.33 -1.44 -1.44 -1.33 -1.33 ...
 $ hum        : num  0.947 0.896 0.896 0.636 0.636 ...
 $ windspeed  : num  -1.55 -1.55 -1.55 -1.55 -1.55 ...
 $ cnt        : int  16 40 32 13 1 1 2 3 8 14 ...
 $ isWorking  : num  0 0 0 0 0 0 0 0 0 0 ...
 $ monthCount : num  1 1 1 1 1 1 1 1 1 1 ...
 $ dayWeek    : num  NA NA NA NA NA NA NA NA NA NA ...
 $ workTime   : num  24 25 26 27 28 29 30 31 32 33 ...
 $ xformHr    : num  19 20 21 22 23 0 1 2 3 4 ...
 $ xformWorkHr: num  43 44 45 46 47 24 25 26 27 28 ...
